# 📊 Мини-проект: Анализ заказов с помощью SQLite и Pandas

In [ ]:

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


## Создание базы данных и таблицы

In [ ]:

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    item TEXT,
    quantity INTEGER,
    price REAL,
    order_date TEXT
)
''')


## Генерация фиктивных данных и вставка в таблицу

In [ ]:

np.random.seed(42)
items = ['Book', 'Laptop', 'Pen', 'Headphones', 'Monitor']
dates = pd.date_range(start='2023-01-01', end='2023-03-31', freq='D')
data = [
    (np.random.randint(1, 10), np.random.choice(items),
     np.random.randint(1, 5), round(np.random.uniform(5, 500), 2),
     pd.to_datetime(np.random.choice(dates)).strftime('%Y-%m-%d'))
    for _ in range(100)
]
cursor.executemany('INSERT INTO orders (user_id, item, quantity, price, order_date) VALUES (?, ?, ?, ?, ?)', data)
conn.commit()


## Выполнение SQL-запросов

In [ ]:

queries = {
    "Общее количество заказов": "SELECT COUNT(*) FROM orders",
    "Общий доход": "SELECT ROUND(SUM(quantity * price), 2) AS revenue FROM orders",
    "Средний чек": "SELECT ROUND(AVG(quantity * price), 2) AS avg_order FROM orders",
    "Самый популярный товар": "SELECT item, COUNT(*) AS freq FROM orders GROUP BY item ORDER BY freq DESC LIMIT 1",
    "Доход по дням": '''
        SELECT order_date, ROUND(SUM(quantity * price), 2) as daily_revenue 
        FROM orders GROUP BY order_date ORDER BY order_date
    '''
}

results = {}
for title, q in queries.items():
    results[title] = pd.read_sql_query(q, conn)
    print(f"--- {title} ---")
    display(results[title])


## Визуализация дохода по дням

In [ ]:

daily = results["Доход по дням"]
daily['order_date'] = pd.to_datetime(daily['order_date'])
plt.figure(figsize=(12, 5))
plt.plot(daily['order_date'], daily['daily_revenue'], marker='o')
plt.title('Доход по дням')
plt.xlabel('Дата')
plt.ylabel('Доход')
plt.grid(True)
plt.tight_layout()
plt.show()


## Закрытие соединения с БД

In [ ]:
conn.close()